This notebook runs the SF election data preparation and analysis

In [10]:
import pandas as pd
import numpy as np
import os
import re
import openpyxl
from xlrd import open_workbook
from datetime import date

from geopandas import GeoDataFrame, read_file

import data_prep_functions as dpf
from spatial_processing_functions import load_prec_shp


import importlib
#importlib.reload(dpf)

## Extract data from election result .xls files

In [11]:
datapath = '../data/SOV_w_nimby/'

# read all files in the folder
# keep data in a dictionary, where keys are election date (e.g., 200111)
file_list = os.listdir(datapath)
vote_data = {}
for f in file_list:
    d=f.strip('SOV').strip('.xls')
    d=d[:-2]
    if d[0]=='9':
        d='19'+d
    else:
        d='20'+d
    vote_data[d]={}
    vote_data[d]['filename'] =f


In [12]:
# define regex search term that will match name of worksheet with local propositions
phrase = 'prop|meas'
phrase_state = 'state|st'

for d in vote_data.keys(): 
    f=vote_data[d]['filename']
    # first only do .xls files.. because need different package to read these.
    if f.split('.')[1]=='xls':      
        #print(f)
        wkbk=open_workbook(datapath+f)
        
        # Find the sheet(s) with local ballot proposals. 
        sheets_w_props=dpf.find_matching_sheets(wkbk, to_match=phrase, to_not_match=phrase_state)
        vote_data[d]['sheet_names'] = sheets_w_props

                
# Here are ones with weird formats. 
vote_data['199711']['sheet_names']=['A - D','E - F']
vote_data['199706']['sheet_names']=['970603']
vote_data['199911']['sheet_names']=['E to H','I to K']

vote_data['201411']['sheet_names']=['370 - Local Measure F']
vote_data['201511']['sheet_names']=['180 - Local Measure D','205 - Local Measure I']



In [13]:
# now find the data within each sheet
# add a list of ballot props we want for each election:

# change this excel file for maybes
proposals = pd.read_excel('../data/BallotPropositions_nimby2.xlsx')  
proposals.head()
proposals['Year_str']=proposals['Year'].astype(str)
proposals['Mo_str']=proposals['Month2'].astype(str)

for i,d in enumerate(proposals['Mo_str']):
    if len(d)==1:
        d='0'+d
    proposals.loc[i,'Mo_str']=d

proposals['Date_str']=proposals['Year_str']+proposals['Mo_str']
#proposals.head()

for d in vote_data.keys():
    p_list=list(proposals[proposals['Date_str']==d]['Letter'].values)
    vote_data[d]['props'] = dict.fromkeys(p_list,{})
#vote_data

In [14]:
# Edit sheetnames so only the ones we need are listed
vote_data['199711']['sheet_names']=['E - F']
vote_data['199811']['sheet_names']= ['City Prop A-E']
vote_data['200011']['sheet_names']=['Prop K-O']
vote_data['200111']['sheet_names']=['AMENDMENTS']

In [15]:
# find the data for that prop in each sheet
for d in vote_data.keys():
    #print(d)
    letters=list(vote_data[d]['props'].keys())
    # we want one sheet for each prop letter

    # I think I should handle them differently if there are multiple props
    # there might be multiple letters, there might be multiple sheets, or both. 
    # but if there's only one letter, there's only one sheet.
    if len(letters) >1:
        # when there are multiple letters, there may be multiple sheets, or just one sheet. 
        # if there are multiple sheets, need to make sure the sheet matches the letter. There are only 3 of these
        # should match these by hand.
        if len(vote_data[d]['sheet_names'])>1:
                print('has multiple sheets:',d)
                
        elif len(vote_data[d]['sheet_names'])<=1:
            # when there are multiple letter but one sheet, easy to match each letter to the one sheet
            #print('has one sheet, multiple letters:',d)
            for l in letters:
                vote_data[d]['props'][l] = {'s_name':vote_data[d]['sheet_names'][0]}

    else:
        # there's only one sheet, which will match the one letter. 
        vote_data[d]['props'][letters[0]]={'s_name':vote_data[d]['sheet_names'][0]}

# where multiple sheets and mult letters, match by hand.
vote_data['201511']['props']['D']={'s_name':'180 - Local Measure D'}
vote_data['201511']['props']['I']={'s_name':'205 - Local Measure I'}
vote_data['199911']['props']['H']={'s_name':'E to H','col_name':'PROP H'}
vote_data['199911']['props']['I']={'s_name':'I to K','col_name':'PROP I'}
vote_data['199911']['props']['J']={'s_name':'I to K','col_name':'PROP J'}
vote_data['201311']['props']['B']={'s_name':'Measure A & B'}
vote_data['201311']['props']['C']={'s_name':'Measure C & D'}

has multiple sheets: 199911
has multiple sheets: 201511
has multiple sheets: 201311


In [16]:
# Huge annoying list of custom parameters needed for reading excel.
d ='200011'
l='K'
vote_data[d]['props'][l]['params']={'index_col':[0,1]}
vote_data[d]['props'][l]['params']['skiprows']=1
vote_data[d]['props'][l]['params']['parse_cols']=[0,1,2,3,5,6]
vote_data[d]['props'][l]['params']['skip_footer']=56

d ='200011'
l='L'
vote_data[d]['props'][l]['params']={'index_col':[0,1]}
vote_data[d]['props'][l]['params']['skiprows']=1
vote_data[d]['props'][l]['params']['parse_cols']=[0,1,2,3,8,9]
vote_data[d]['props'][l]['params']['skip_footer']=56

d ='200111'
l='D'
vote_data[d]['props'][l]['params']={'index_col':[0,1]}
vote_data[d]['props'][l]['params']['skiprows']=1
vote_data[d]['props'][l]['params']['parse_cols']=[0,1,2,3,7,8]
vote_data[d]['props'][l]['params']['skip_footer']=70

d ='199911'
l='J'
vote_data[d]['props'][l]['params']={'index_col':[0,1]}
vote_data[d]['props'][l]['params']['skiprows']=4
vote_data[d]['props'][l]['params']['parse_cols']=[0,1,2,3,9,10]
vote_data[d]['props'][l]['params']['skip_footer']=51

d ='199911'
l='H'
vote_data[d]['props'][l]['params']={'index_col':[0,1]}
vote_data[d]['props'][l]['params']['skiprows']=4
vote_data[d]['props'][l]['params']['parse_cols']=[0,1,2,3,15,16]
vote_data[d]['props'][l]['params']['skip_footer']=51

d ='199911'
l='I'
vote_data[d]['props'][l]['params']={'index_col':[0,1]}
vote_data[d]['props'][l]['params']['skiprows']=4
vote_data[d]['props'][l]['params']['parse_cols']=[0,1,2,3,6,7]
vote_data[d]['props'][l]['params']['skip_footer']=51

d ='200203'
l='D'
vote_data[d]['props'][l]['params']={'index_col':[0,1]}
vote_data[d]['props'][l]['params']['skiprows']=1
vote_data[d]['props'][l]['params']['parse_cols']=[0,1,2,3,23,24]
vote_data[d]['props'][l]['params']['skip_footer']=60

d ='200806'
l='F'
vote_data[d]['props'][l]['params']={'index_col':0}
vote_data[d]['props'][l]['params']['skiprows']=2
vote_data[d]['props'][l]['params']['parse_cols']=[0,1,2,24,25]
vote_data[d]['props'][l]['params']['skip_footer']=51

d ='200806'
l='G'
vote_data[d]['props'][l]['params']={'index_col':0}
vote_data[d]['props'][l]['params']['skiprows']=2
vote_data[d]['props'][l]['params']['parse_cols']=[0,1,2,28,29]
vote_data[d]['props'][l]['params']['skip_footer']=51

d ='199711'
l='H'
vote_data[d]['props'][l]['params']={'index_col':0}
vote_data[d]['props'][l]['params']['skiprows']=3
vote_data[d]['props'][l]['params']['parse_cols']=[0,1,2,14,15]
vote_data[d]['props'][l]['params']['skip_footer']=42

d ='199603'
l='B'
vote_data[d]['props'][l]['params']={'index_col':0}
vote_data[d]['props'][l]['params']['skiprows']=3
vote_data[d]['props'][l]['params']['parse_cols']=[0,1,2,8,9]
vote_data[d]['props'][l]['params']['skip_footer']=38

d ='201511'
l='I'
vote_data[d]['props'][l]['params']={'index_col':[0,1]}
vote_data[d]['props'][l]['params']['skiprows']=3
vote_data[d]['props'][l]['params']['parse_cols']=[0,1,4,5,7,8]
vote_data[d]['props'][l]['params']['skip_footer']=54

d ='201511'
l='D'
vote_data[d]['props'][l]['params']={'index_col':[0,1]}
vote_data[d]['props'][l]['params']['skiprows']=3
vote_data[d]['props'][l]['params']['parse_cols']=[0,1,4,5,7,8]
vote_data[d]['props'][l]['params']['skip_footer']=54

d ='200211'
l='R'
vote_data[d]['props'][l]['params']={'index_col':[0,1]}
vote_data[d]['props'][l]['params']['skiprows']=1
vote_data[d]['props'][l]['params']['parse_cols']=[0,1,2,3,53,54]
vote_data[d]['props'][l]['params']['skip_footer']=64

d ='200211'
l='B'
vote_data[d]['props'][l]['params']={'index_col':[0,1]}
vote_data[d]['props'][l]['params']['skiprows']=1
vote_data[d]['props'][l]['params']['parse_cols']=[0,1,2,3,21,22]
vote_data[d]['props'][l]['params']['skip_footer']=64

d ='201406'
l='B'
vote_data[d]['props'][l]['params']={'index_col':0}
vote_data[d]['props'][l]['params']['skiprows']=0
vote_data[d]['props'][l]['params']['parse_cols']=[6,7,8,14,15]
vote_data[d]['props'][l]['params']['skip_footer']=55

d ='200411'
l='A'
f_name=vote_data[d]['filename']
s=vote_data[d]['props'][l]['s_name']

vote_data[d]['props'][l]['params']={'index_col':[0,1]}
vote_data[d]['props'][l]['params']['skiprows']=2
vote_data[d]['props'][l]['params']['parse_cols']=[0,1,2,3,5,6]
vote_data[d]['props'][l]['params']['skip_footer']=32

d ='201311'
l='C'
vote_data[d]['props'][l]['params']={'index_col':0}
vote_data[d]['props'][l]['params']['skiprows']=1
vote_data[d]['props'][l]['params']['parse_cols']=[0,1,2,4,5]
vote_data[d]['props'][l]['params']['skip_footer']=54

d ='201311'
l='B'
vote_data[d]['props'][l]['params']={'index_col':0}
vote_data[d]['props'][l]['params']['skiprows']=1
vote_data[d]['props'][l]['params']['parse_cols']=[0,1,2,8,9]
vote_data[d]['props'][l]['params']['skip_footer']=54

d ='200611'
l='G'
vote_data[d]['props'][l]['params']={'index_col':[0,1]}
vote_data[d]['props'][l]['params']['skiprows']=1
vote_data[d]['props'][l]['params']['parse_cols']=[0,1,2,3,17,18]
vote_data[d]['props'][l]['params']['skip_footer']=33

d ='199706'
l='F'
vote_data[d]['props'][l]['params']={'index_col':0}
vote_data[d]['props'][l]['params']['skiprows']=3
vote_data[d]['props'][l]['params']['parse_cols']=[0,1,2,20,21]
vote_data[d]['props'][l]['params']['skip_footer']=38

d ='200003'
l='C'
vote_data[d]['props'][l]['params']={'index_col':[0,1]}
vote_data[d]['props'][l]['params']['skiprows']=5
vote_data[d]['props'][l]['params']['parse_cols']=[0,1,2,3,12,13]
vote_data[d]['props'][l]['params']['skip_footer']=52

d ='201411'
l='F'
vote_data[d]['props'][l]['params']={'index_col':[0,1]}
vote_data[d]['props'][l]['params']['skiprows']=3
vote_data[d]['props'][l]['params']['parse_cols']=[0,1,4,5,7,8]
vote_data[d]['props'][l]['params']['skip_footer']=54

d ='199811'
l='E'
vote_data[d]['props'][l]['params']={'index_col':0}
vote_data[d]['props'][l]['params']['skiprows']=3
vote_data[d]['props'][l]['params']['parse_cols']=[0,1,2,17,18]
vote_data[d]['props'][l]['params']['skip_footer']=38

d ='199806'
l='E'
vote_data[d]['props'][l]['params']={'index_col':0}
vote_data[d]['props'][l]['params']['skiprows']=5
vote_data[d]['props'][l]['params']['parse_cols']=[0,1,2,17,18]
vote_data[d]['props'][l]['params']['skip_footer']=35

d ='199806'
l='K'
vote_data[d]['props'][l]['params']={'index_col':0}
vote_data[d]['props'][l]['params']['skiprows']=5
vote_data[d]['props'][l]['params']['parse_cols']=[0,1,2,35,36]
vote_data[d]['props'][l]['params']['skip_footer']=35

d ='199806'
l='I'
vote_data[d]['props'][l]['params']={'index_col':0}
vote_data[d]['props'][l]['params']['skiprows']=5
vote_data[d]['props'][l]['params']['parse_cols']=[0,1,2,29,30]
vote_data[d]['props'][l]['params']['skip_footer']=35

d ='200403'
l='J'
vote_data[d]['props'][l]['params']={'index_col':[0,1]}
vote_data[d]['props'][l]['params']['skiprows']=1
vote_data[d]['props'][l]['params']['parse_cols']=[0,1,2,3,33,34]
vote_data[d]['props'][l]['params']['skip_footer']=65

d ='200411'
l='A'
vote_data[d]['props'][l]['params']={'index_col':[0,1]}
vote_data[d]['props'][l]['params']['skiprows']=2
vote_data[d]['props'][l]['params']['parse_cols']=[0,1,2,3,5,6]
vote_data[d]['props'][l]['params']['skip_footer']=65
#Done!!


In [17]:
# process data... 
dlist=list(vote_data.keys())

# read excel files for each election... 
for d in dlist:
    print(d)
    for l in vote_data[d]['props'].keys():
        print(l)
        data=dpf.read_vote_sheet(d,l,vote_df=vote_data, path='../data/SOV_w_nimby/')
        # test if already has multiindex:
        if isinstance(data.index, pd.core.index.MultiIndex): 
            data=dpf.rename_index_and_cols(data)
        else:
            desc = dpf.check_if_descriptive(d)
            data=dpf.format_df_to_multiindex(data, descriptive_labels=desc)
            data=dpf.rename_index_and_cols(data)
        vote_data[d]['props'][l]['data']=data
#data.head()

199706
F
200611
G
201406
B
199911
H
I
J
200211
R
B
200003
C
200411
A
199711
H
200011
L
K
200403
J
200111
D
199806
K
E
I
201511
D
I
200806
F
G
199811
E
199603
B
201311
C
B
201411
F
200203
D


In [18]:
# Verify totals

dpf.verify_vote_totals(vote_data, fname='verify_percentages.csv')

# Check sums verify. Some of these are off. 
# For the 90s elections, there are three mysterious "ballot types" at the beginning that don't have a precinct.
# that's what's throwing off the totals. Might be votes for people who don't vote at an address, like prisoners or something.
# So just make sure that, when reporting full results, rely on the original sheet and not the dataframe. 
# otherwise everything checks out. 
   

In [19]:
vote_data.keys()
test_df = vote_data['200403']['props']['J']['data']
test_df.head()

registered  voted  YES   NO
precinct type                             
PCT 1101 A            766    122   35   79
         V            766    173   41  121
PCT 1102 A            921    166   41  110
         V            921    251   66  167
PCT 1103 A            783    125   50   59

## Process data

In [20]:

vote_data1 = dpf.process_votedata(vote_data, dpf.consolidate_abs)
vote_data1.keys()
vote_data1['200806']['props']['G']['data'].head()

,voted,YES,NO,registered
precinct,,,,
PCT 1101,315,201,90,806
PCT 1102,397,247,115,937
PCT 1103,283,169,90,750
PCT 1104,335,198,112,844
PCT 1105,282,161,92,884


## define variables

In [21]:

vote_data2 = dpf.process_votedata(vote_data1, dpf.make_vote_variables, use_datekey=True, use_propkey=True,df_prop=proposals)

vote_data2['201511']['props']['I']['data'].head()

,voted,YES,NO,registered,tot_nimby_votes,pct_nimby,turnout
precinct,,,,,,,
Pct 1101,210,87,109,492,87,0.443878,0.426829
Pct 1102/1103,817,319,466,1748,319,0.406369,0.467391
Pct 1104/1105,376,191,167,863,191,0.533520,0.435689
Pct 1106/1107,571,240,307,1413,240,0.438757,0.404105
Pct 1109/1108,699,284,385,1627,284,0.424514,0.429625


## Merge voting data with census data by precinct

In [22]:

vote_data3 = dpf.process_votedata(vote_data2, dpf.format_precincts)

# let's merge!
vote_data4 = dpf.process_votedata(vote_data3, dpf.merge_vote_census, use_datekey=True)
# All matched up. 


 for election  199706
length vote data:  534 length census data:  712
length new data:  533

 for election  200611
length vote data:  581 length census data:  586
length new data:  562

 for election  201406
length vote data:  597 length census data:  603
length new data:  597

 for election  199911
length vote data:  687 length census data:  712
length new data:  680

 for election  199911
length vote data:  687 length census data:  712
length new data:  680

 for election  199911
length vote data:  687 length census data:  712
length new data:  680

 for election  200211
length vote data:  713 length census data:  712
length new data:  691

 for election  200211
length vote data:  713 length census data:  712
length new data:  691

 for election  200003
length vote data:  688 length census data:  712
length new data:  681

 for election  200411
length vote data:  581 length census data:  586
length new data:  564

 for election  199711
length vote data:  599 length census data:  712

In [23]:
df_test = vote_data4['200403']['props']['J']['data']
df_test.columns

Index(['voted', 'YES', 'NO', 'registered', 'tot_nimby_votes', 'pct_nimby',
       'turnout', 'precname', 'area_m', 'tot_hu_wgt', 'occ_hu_wgt',
       'owned_wgt', 'rented_wgt', 'hu_detatched_wgt', 'hu_2_wgt', 'hu_3-4_wgt',
       'hu_5-9_wgt', 'hu_10-19_wgt', 'hu_20-49_wgt', 'hu_50_wgt',
       'families_wgt', 'tot_hhs_wgt', 'tot_pop_wgt', 'white_wgt', 'black_wgt',
       'asian_wgt', 'hispanic_wgt', 'med_yr_built_wgt', 'foreign_born_wgt',
       'med_yr_moved_all_wgt', 'med_yr_moved_owner_wgt', 'med_value_wgt',
       'med_inc_wgt', 'med_age_wgt'],
      dtype='object')

# Prepare data for R, then save

Make a dataset that combines all elections, but make sure to have proposal and year dummies

Create dummy variables:  
- year

- pres election year

- november election
 

In [24]:
vote_data5 = dpf.process_votedata(vote_data4, dpf.make_election_dummies, use_datekey=True, use_propkey=True)
vote_data6 = dpf.process_votedata(vote_data5, dpf.fix_yr_built_moved, use_datekey=True)
vote_data7 = dpf.process_votedata(vote_data6, dpf.adjust_inflation, use_datekey=True)

In [25]:
df_test = vote_data7['200403']['props']['J']['data']
df_test.head()

,voted,YES,NO,registered,tot_nimby_votes,pct_nimby,turnout,precname,area_m,tot_hu_wgt,...,yr_2004,pres_elec,nov_elec,year,yr_prop,med_hu_age,med_yrs_lived,med_yrs_lived_owner,med_inc_adj,med_val_adj
0,295,76,200,766,200,0.724638,0.385117,1101,2696087.189278,542.163803,...,True,False,False,2004,200403J,64.806870,8.188377,8.742502,103635.537362,692651.358909
1,417,107,277,921,277,0.721354,0.452769,1102,1073412.658539,422.709863,...,True,False,False,2004,200403J,63.226691,8.327617,8.984652,93123.572423,701135.973615
2,250,81,143,783,143,0.638393,0.319285,1103,2463865.571322,467.354006,...,True,False,False,2004,200403J,60.671727,11.145154,14.160339,74442.411493,627709.949953
3,413,112,271,902,271,0.707572,0.457871,1104,1664514.050366,468.618337,...,True,False,False,2004,200403J,64.981444,4.991931,11.144900,103307.967471,742403.924155
4,324,84,205,910,205,0.709343,0.356044,1105,2383380.691068,286.896614,...,True,False,False,2004,200403J,64.349427,5.123913,13.337303,99354.103620,744537.791721


In [26]:
  
all_data = dpf.combine_dataframes(vote_data7)

working on  199706 F
533
working on  200611 G
562
working on  201406 B
597
working on  199911 H
680
working on  199911 I
680
working on  199911 J
680
working on  200211 R
691
working on  200211 B
691
working on  200003 C
681
working on  200411 A
564
working on  199711 H
598
working on  200011 L
688
working on  200011 K
688
working on  200403 J
581
working on  200111 D
684
working on  199806 K
643
working on  199806 E
643
working on  199806 I
643
working on  201511 D
582
working on  201511 I
582
working on  200806 F
561
working on  200806 G
561
working on  199811 E
643
working on  199603 B
649
working on  201311 C
409
working on  201311 B
409
working on  201411 F
582
working on  200203 D
688


In [27]:
# fill na with False, for year dummies
yr_cols = ['yr_1996', 'yr_1997', 'yr_1998','yr_1999', 'yr_2000', 'yr_2001', 'yr_2002', 'yr_2004', 'yr_2006',
       'yr_2008', 'yr_2013', 'yr_2014', 'yr_2015']

all_data[yr_cols] = all_data[yr_cols].fillna(value=False)

# rename columns to get rid of "_wgt"
all_data = dpf.rename_columns(all_data)


In [62]:
# save!

date.today().strftime('%m%d%Y')
filepath = '../results/'
d = date.today().strftime('%m%d%Y')
filename = 'voting_data_all_{}.csv'.format(d)

all_data.to_csv(filepath+filename, index=False)


# Prepare data for visualization

- Join result data with precincts, on prec ID, matching appropriate years.

- Write to geojson files - 1 geojson file for each precinct layer (3 files)

In [50]:
# make a column with year_month
all_data['yr_mo'] = all_data['yr_prop'].str[:-1].astype(int)

prec_keys=['pre1992','pre2002','pre2012']

for prec_key in prec_keys:
    yr = prec_key[-4:]
    
    # load precinct shapefiles as geodataframes
    prec_df = load_prec_shp(yr)
    
    # filter all data based on the year
    vote_df = filter_for_dates(all_data, prec_key)  
    
    # merge on precinct id 
    merged = pd.merge(prec_df, vote_df, on='precname')
    
    # remove unneeded columns to reduce file size
    cols_to_drop = ['foreign_born','yr_1996', 'yr_1997', 'yr_1998','yr_1999', 'yr_2000', 'yr_2001', 'yr_2002', 'yr_2004', 'yr_2006',
       'yr_2008', 'yr_2013', 'yr_2014', 'yr_2015']
    try: 
        merged = merged.drop(cols_to_drop, axis=1)
    except ValueError as err:
        print('Error: {}'.format(err))
    
    # write to geojson by writing to a json string
    path= '../results/maps/'
    filename = 'results_{}.geojson'.format(prec_key)
    
    with open(path+filename, 'w') as f:
        f.write(merged.to_json())
        
    